In [1]:
import tensorflow as tf
import numpy as np
from models._48_net import detect_48Net
from models._24_net import calib_24Net
from utils.params import *
from utils.sampling import GetSample
from utils import save
from utils.utilities import *
import sys
import pickle

In [2]:
get = GetSample()
pos_db_48 = get.get_pos_img(p_net_48)
save.saver('pos_db_48.txt', pos_db_48)

Samples Checked..
IMG_INDEX:112/112
Finished


In [2]:
inputs = np.zeros((p_batch_size ,p_net_48, p_net_48, 3))
targets = np.zeros((p_batch_size, 1))
pos_db_48 = save.loader('pos_db_48.txt')
neg_db_48 = save.loader('neg_db_48_filter.txt')
valid_db_48 = save.loader('valid_db_48.txt')

Loaded from D:\programing project\python project\trying stuffpickle/
Loaded from D:\programing project\python project\trying stuffpickle/
Loaded from D:\programing project\python project\trying stuffpickle/


In [3]:
valid_input = np.array([arr[0] for arr in valid_db_48])
valid_target = np.array([arr[1] for arr in valid_db_48])

In [4]:
def detnet_evaluation(pred_arr, true_arr, thres=.5):
    pred_idx = np.where(pred_arr>thres)[0]
    pred_tf = (pred_arr>thres)*1
    #print(len(pred_idx))
    correct_idx = len(np.where(true_arr[pred_idx] == 1)[0]) # Predicted Positives that are true positive
    true_len = np.sum(true_arr == 1.0)
    
    correct_numb = np.sum((pred_tf==true_arr))
    return correct_idx / true_len, correct_idx/(len(pred_idx)+1e-7)

def f1_score(recall, precision, weight=1.0):
    a = weight**2
    return (a + 1)*recall*precision / a*(recall + precision)

In [5]:
np.sum(valid_target == 1.0)

60

In [6]:
bias = [0.0]
lr = [1e-3, 5e-4, 1e-4, 5e-5]
reg = [0.0, 1e-3]
epochs = 500
neg_db_48 = neg_db_48[:10080,:]
batch_len = int(len(neg_db_48) / p_neg_batch)
total_iters = batch_len * epochs

pickle_path = 'saver/det_48/mean_var/'

In [ ]:
for r in reg:
    for b in bias:
        for l in lr:
            record_l = l
            print('\n**Bias_init:{}   Learning_rate:{}   Regularization:{}**'.format(b,l,r))
            
            # initialize Batch selection function
            batch_sel_pos = batch_selection()
            batch_sel_neg = batch_selection()
            
            bad_cnt = 0
            max_epo = 0
            max_true_acc = 0
            max_recall = 0
            max_f1 = 0
            
            # Build the 48_det_CNN
            g = tf.Graph()
            with g.as_default():
                input_48_node = tf.placeholder(tf.float32, [None,None,None,3])
                target_48_node = tf.placeholder(tf.float32, [None,1])
                keep_prob = tf.placeholder(tf.float32)
                det_net_48 = detect_48Net(input_48_node, target_48_node, keep_prob=keep_prob, lr=l, bias_init=b, reg=r)
                
                sess = tf.InteractiveSession(config=tf.ConfigProto(
                  allow_soft_placement=True, log_device_placement=True), graph=g)
                sess.run(tf.global_variables_initializer())
            
            total_mean_var_list = np.zeros((4,64))
            offset_scale_arr = np.zeros((4,64))
            
            pred_arr = det_net_48.h_fc_conv2_flat.eval(feed_dict={input_48_node:valid_input, keep_prob:1.0})
            recall, true_acc = detnet_evaluation(pred_arr, valid_target, .5)
            f1 = f1_score(recall, true_acc)
            
            for epo in range(epochs):
                loss = 0
                
                if (epo+1) % 2 == 0:
                    pred_arr = det_net_48.h_fc_conv2_flat.eval(feed_dict={input_48_node:valid_input, keep_prob:1.0})
                    recall, true_acc = detnet_evaluation(pred_arr, valid_target, .5)
                    f1 = f1_score(recall, true_acc)
                    
                    if f1 >= max_f1:

                        max_true_acc = true_acc
                        max_recall = recall
                        max_f1 = f1

                        max_epo = epo
                        bad_cnt = 0
                        saver = tf.train.Saver()
                        saver.save(sess, p_model_dir + 'det_48/max/_Net48_b={}_lr={}_reg={}.ckpt'.format(b,record_l,r))
                        
                        with open(pickle_path + '_Net48_MV_b={}_lr={}_reg={}.txt'.format(b, record_l, r), 'wb') as f:
                            pickle.dump(mean_var_list, f)
                    else:
                        bad_cnt += 1
                        
                if bad_cnt % 4 == 0:
                    l = l * .98
                    
                if bad_cnt > 50:
                    break
                
                mean_var_list = np.zeros((4,64))
                
                
                for i in range(batch_len):
                    cur_iter = i + batch_len * epo
                    
                    inputs[:p_pos_batch,:] = batch_sel_pos.next_batch_single(pos_db_48, p_pos_batch)
                    targets[:p_pos_batch,:] = np.ones((p_pos_batch, 1))
                    inputs[p_pos_batch:,:] = batch_sel_neg.next_batch_single(neg_db_48, p_neg_batch)
                    targets[p_pos_batch:,:] = np.zeros((p_neg_batch, 1))

                    feed = {
                        input_48_node:inputs,
                        target_48_node:targets,
                        keep_prob: 1.0
                    }
                    
                    feed_train = {
                        input_48_node:inputs,
                        target_48_node:targets,
                        keep_prob: .5
                    }
                    
                    
                    
                    loss += det_net_48.loss.eval(feed)
                    single_loss = det_net_48.loss.eval(feed)

                    det_net_48.train_step.run(feed_train)
                    
                    mean_var_list[0,:] += det_net_48.mean_wconv1.eval(feed)
                    mean_var_list[1,:] += det_net_48.var_wconv1.eval(feed)
                    mean_var_list[2,:] += det_net_48.mean_wconv2.eval(feed)
                    mean_var_list[3,:] += det_net_48.var_wconv2.eval(feed)

                    sys.stdout.write('\rIteration: ' + str(cur_iter+1) + '/' + str(total_iters) + \
                                     '    Loss: ' + str(loss/(cur_iter+1))[:7] + '/' + str(single_loss) + \
                                     '    Bad: ' + str(bad_cnt) + \
                                     '    R/AC/F1: ' + str(recall*100)[:6] + '%' + '/' + str(true_acc*100)[:6] + '%/' + \
                                     str(f1*100)[:6] + \
                                     '    MAX_R/AC/F1: ' + str(max_recall*100)[:6] + '%' + '/' + str(max_true_acc*100)[:6] + '%/' + \
                                     str(max_f1*100)[:6] + \
                                     '   Max_Epo: ' + str(max_epo))
                mean_var_list = mean_var_list / batch_len
            
            #offset_scale_arr[0,:] = det_net_48.of_conv1.eval()
            #offset_scale_arr[1,:] = det_net_48.sc_conv1.eval()
            #offset_scale_arr[2,:] = det_net_48.of_conv2.eval()
            #offset_scale_arr[3,:] = det_net_48.sc_conv2.eval()
            
            #total_mean_var_list = total_mean_var_list / epochs


**Bias_init:0.0   Learning_rate:0.001   Regularization:0.0**
Iteration: 122/17500    Loss: 0.01141/0.0733929    Bad: 0    R/AC/F1: 41.666%/86.206%/91.863    MAX_R/AC/F1: 41.666%/86.206%/91.863   Max_Epo: 3

In [19]:
batch_sel_pos = batch_selection()
batch_sel_neg = batch_selection()
tmean_var_list = np.zeros((4,64))
for i in range(batch_len):
    inputs[:p_pos_batch,:] = batch_sel_pos.next_batch_single(pos_db_48, p_pos_batch)
    targets[:p_pos_batch,:] = np.ones((p_pos_batch, 1))
    inputs[p_pos_batch:,:] = batch_sel_neg.next_batch_single(neg_db_48, p_neg_batch)
    targets[p_pos_batch:,:] = np.zeros((p_neg_batch, 1))
    feed = {
            input_48_node:inputs,
            target_48_node:targets,
            keep_prob: 1.0
        }
    tmean_var_list[0,:] += det_net_48.mean_wconv1.eval(feed)
    tmean_var_list[1,:] += det_net_48.var_wconv1.eval(feed)
    tmean_var_list[2,:] += det_net_48.mean_wconv2.eval(feed)
    tmean_var_list[3,:] += det_net_48.var_wconv2.eval(feed)
    
tmean_var_list = tmean_var_list / batch_len

In [20]:
tmean_var_list[0]

array([  2.52908495e-01,   4.61446481e-01,   9.81779911e-01,
         7.76999937e-01,   8.37855125e-03,   1.85452061e-01,
         4.95153850e-01,   3.40492738e-01,   3.32354470e-03,
         1.45554514e-02,   1.93674001e-01,   1.45450069e-01,
         1.60463558e+00,   4.32460479e-01,   2.84323978e-01,
         1.82362037e-01,   2.59806626e-02,   1.41569671e-01,
         2.36387303e-01,   2.73599435e-01,   1.29298050e-02,
         4.42843865e-02,   2.44863621e-02,   1.81766756e-02,
         8.85730882e-02,   1.30457179e-02,   2.77885529e-01,
         1.72764273e+00,   2.16383407e-01,   1.62746808e-01,
         1.09626971e+00,   1.08306128e-01,   3.91063710e-01,
         1.36166664e+00,   7.54949956e-01,   1.46540683e-01,
         1.02859631e-03,   1.83600186e-01,   1.24208521e-03,
         1.34620617e-01,   1.62193016e-02,   7.74461877e-01,
         1.12280480e+00,   1.97339045e-01,   1.40744009e-03,
         5.69657329e-02,   2.31870877e-01,   5.67934128e-02,
         8.20592523e-02,

In [21]:
mean_var_list[0]

array([  2.50960941e-01,   4.63285848e-01,   9.82836609e-01,
         7.77118596e-01,   8.37059833e-03,   1.85686649e-01,
         4.93709628e-01,   3.37909466e-01,   3.25630203e-03,
         1.44065173e-02,   1.93330073e-01,   1.43831174e-01,
         1.60672680e+00,   4.33834830e-01,   2.88563408e-01,
         1.83027148e-01,   2.59973982e-02,   1.41912745e-01,
         2.36671602e-01,   2.75211393e-01,   1.28892390e-02,
         4.38240043e-02,   2.45335301e-02,   1.82645093e-02,
         8.72009531e-02,   1.29934340e-02,   2.78458254e-01,
         1.72943207e+00,   2.15405279e-01,   1.58615994e-01,
         1.09639059e+00,   1.07906934e-01,   3.96223021e-01,
         1.36200685e+00,   7.56211868e-01,   1.45128296e-01,
         1.02876989e-03,   1.81334391e-01,   1.24890130e-03,
         1.31621446e-01,   1.61902729e-02,   7.76542032e-01,
         1.12573348e+00,   1.95331793e-01,   1.39888260e-03,
         5.64789913e-02,   2.31226784e-01,   5.67178888e-02,
         8.27340177e-02,

In [8]:
g = tf.Graph()
with g.as_default():
    input_48_node = tf.placeholder(tf.float32, [None,None,None,3])
    target_48_node = tf.placeholder(tf.float32, [None,1])
    keep_prob = tf.placeholder(tf.float32)
    det_net_48 = detect_48Net(input_48_node, target_48_node, keep_prob=keep_prob, lr=l, bias_init=b, reg=r)

    sess = tf.InteractiveSession(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True), graph=g)
    sess.run(tf.global_variables_initializer())

In [9]:
feed={
    input_48_node:np.zeros((1,48,48,3))
}

In [10]:
det_net_48.mean_wconv1.eval(feed).shape

(64,)

In [27]:
img = tf.Variable(tf.random_normal([128, 32, 32, 64]))
axis = list(range(len(img.get_shape()) - 1))


In [28]:
axis

[0, 1, 2]